# **Análise de Crédito e Inadimplência**

Projeto final do módulo ‘Python para Análise de Dados’, parte integrante do curso de Ciência de Dados da EBAC.


# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Exploração dos dados;</li>
  <li>Transformação e limpeza dos dados;</li>
  <li>Visualização dos dados;</li>
  <li>Resultados.</li>
</ol>


# 1\. Introdução

O objetivo desta análise é investigar os fatores que contribuem para a inadimplência de um cliente, com base em seu comportamento e outros atributos, como salário, escolaridade e movimentação financeira.

Para isso, vamos explorar os dados de crédito disponíveis no seguinte [link](https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv). Esses dados estão em formato CSV e fornecem informações sobre os clientes de uma instituição financeira. (Fornecidos por [André Perez](https://www.linkedin.com/in/andremarcosperez/))

Especificamente, concentraremos nossa atenção na segunda coluna, denominada "default", que indica se um cliente é adimplente (default = 0) ou inadimplente (default = 1).

Abaixo está uma descrição completa dos atributos presentes no  arquivo CSV:

| Coluna  | Descrição |
| ------- | --------- |
| id      | Número da conta |
| default | Indica se o cliente é adimplente (0) ou inadimplente (1) |
| idade   | Idade do cliente |
| sexo    | Gênero do cliente |
| depedentes | Número de dependentes financeiros |
| escolaridade | Nível de escolaridade |
| estado_civil | Estado civil |
| salario_anual | Faixa do salario mensal multiplicado por 12 |
| tipo_cartao | Categoria do cartao: blue, silver, gold e platinium |
| meses_de_relacionamento | Quantidade de meses desde a abertura da conta |
| qtd_produtos | Quantidade de produtos contratados |
| iteracoes_12m | Quantidade de iteracoes com o cliente no último ano |
| meses_inatico_12m | Quantidade de meses que o cliente ficou inativo no último ano |
| limite_credito | Valor do limite do cartão de crédito |
| valor_transacoes_12m | Soma total do valor das transações no cartão de crédito no último ano |
| qtd_transacoes_12m | Quantidade total de transações no cartão de crédito no último ano |



# 2\. Exploração dos Dados

Vamos iniciar importando as bibliotecas necessárias e realizando a leitura dos dados.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv', na_values='na')
df.head(10)

## 2\.1 Schema

Vamos examinar a estrutura e o formato dos dados

In [ ]:
linhas, colunas = df.shape
print(f'O arquivo possui {linhas} linhas e {colunas} colunas.')

In [ ]:
total_clientes = len(df)
adimplentes = df['default'].value_counts()[0]
inadimplentes = df['default'].value_counts()[1]

prop_adimplentes = round(100 * adimplentes / total_clientes, 2)
prop_inadimplentes = round(100 * inadimplentes / total_clientes, 2)

print(f'A proporção de clientes adimplentes é de {prop_adimplentes}% ({adimplentes} de {total_clientes})')
print(f'A proporção de clientes inadimplentes é de {prop_inadimplentes}% ({inadimplentes} de {total_clientes})')


In [ ]:
df.columns # lista com o nome das colunas

In [ ]:
df.dtypes # tipos de dados de cada coluna

In [ ]:
df.describe().drop('id', axis=1) # estatísticas descritivas dos atributos numéricos

In [ ]:
df.describe(include=['object']) # estatísticas descritivas dos atributos categóricos

## 2\.2 Dados faltantes

A seguir, apresentaremos algumas estatísticas sobre os dados ausentes em colunas específicas.

In [ ]:
df.isna().any() # indica se há valores ausentes

In [ ]:
def total_faltantes(df: pd.DataFrame) -> None:
    qtd_total = df.isna().sum().sum()
    prop_total = df.isna().mean().mean() * 100
    print(f"A base de dados apresenta um total de {qtd_total} dados faltantes ({prop_total:.2f}%)")

In [ ]:
total_faltantes(df) # quantidade total de valores faltantes

In [ ]:
def dados_faltantes(df):
    missing_data = df.isna().sum()
    missing_percentage = 100 * df.isna().mean()

    for col, qtd, prop in zip(df.columns, missing_data, missing_percentage):
        if qtd > 0:
            print(f'{col}: faltam {qtd} dados ({prop:.2f}%)')

In [ ]:
dados_faltantes(df) # apresenta os dados faltantes por coluna

In [ ]:
print('Proporção de dados faltantes dos clientes adimplentes:')
dados_faltantes(df[df['default'] == 0])

In [ ]:
print('Proporção de dados faltantes dos clientes inadimplentes:')
dados_faltantes(df[df['default'] == 1])

# 3\. Transformação e limpeza dos dados

Vamos corrigir as colunas "valor_transacoes_12m" e "limite_credito", que estão sendo erroneamente interpretadas como categóricas, e em seguida, remover os dados faltantes.

In [ ]:
def conv_moeda(df, column):
    df[column] = df[column].str.replace('.', '').str.replace(',', '.').astype(float)


Como constatamos que a proporção de linhas com dados faltantes não tem grande impacto em nossa base de dados, podemos excluí-las.

In [ ]:
conv_moeda(df, 'limite_credito')
conv_moeda(df, 'valor_transacoes_12m')
df.dropna(inplace=True)

In [ ]:
print(df.info()) # informações gerais sobre as mudanças realizadas

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].head()

In [ ]:
print(f"O total de linhas agora é: {len(df)}")


In [ ]:
qtd_total_novo = len(df)
qtd_adimplentes_novo, qtd_inadimplentes_novo = df.groupby('default').size()

In [ ]:
prop_adimplentes_novo = round(100 * qtd_adimplentes_novo / qtd_total_novo, 2)
prop_inadimplentes_novo = round(100 * qtd_inadimplentes_novo / qtd_total_novo, 2)

print(f'A nova proporção de clientes adimplentes é de {prop_adimplentes_novo}% ({qtd_adimplentes_novo} de {qtd_total_novo})')
print(f'A nova proporção de clientes inadimplentes é de {prop_inadimplentes_novo}% ({qtd_inadimplentes_novo} de {qtd_total_novo})')

# 4\. Visualização dos dados

In [ ]:
df.corr()

# 5\. Resultados